### Ladi_ai_RAG

- API DOCS : http://43.202.173.71:3000/
- TASK : 루틴 추천
- 루틴 관련 데이터 저장(vectorDB) -> 검색(Retrieve) -> LLM(gpt-4o)의 Context로 사용

In [ ]:
import requests
import json

# API 엔드포인트 URL
api_url = "http://43.202.173.71:3000/api/endpoint"  # 실제 엔드포인트 URL로 변경해야 합니다

# 요청 데이터
request_data = {
    "Difficult": "의욕 떨어짐, 목표 너무 큼",
    "Job": "학생",
    "gender": "남성",
    "age": 24,
    "goal": "꾸준히 운동하기"
}

# POST 요청 보내기
response = requests.post(api_url, json=request_data)

# 응답 처리
if response.status_code == 200:
    # 성공적인 응답
    response_data = response.json()
    print("API 응답:")
    print(json.dumps(response_data, indent=2, ensure_ascii=False))
else:
    # 오류 응답
    print(f"오류 발생: {response.status_code}")
    print(response.text)

In [8]:
import requests

# API 엔드포인트 URL
token_url = "http://43.202.173.71:3000/auth/token"

# 요청 데이터 (필요한 경우 수정)
request_data = {
    "username": "your_username",  # 사용자 이름
    "password": "your_password"   # 비밀번호
}

# POST 요청 보내기
response = requests.post(token_url)

# 응답 처리
if response.status_code == 200:
    # 성공적인 응답
    token_data = response.json()
    print("토큰 응답:")
    print(json.dumps(token_data, indent=2, ensure_ascii=False))
else:
    # 오류 응답
    print(f"오류 발생: {response.status_code}")
    print(response.text)


오류 발생: 401
Unauthorized


In [6]:
import requests
# response=requests.get('http://43.202.173.71:3000/statistics')
# print(response.json())

response = requests.get('http://43.202.173.71:3000/statistics')
if response.status_code == 200:
    try:
        data = response.json()
    except ValueError:
        print("응답이 JSON 형식이 아닙니다:", response.text)
else:
    print(f"오류 발생: {response.status_code} - {response.text}")


오류 발생: 401 - Unauthorized


In [ ]:
from flask import Flask, jsonify
import requests

app = Flask(__name__)

@app.route('/get_statistics', methods=['GET'])
def get_statistics():
    try:
        # localhost:3000/statistics에 GET 요청 보내기
        response = requests.get('http://43.202.173.71:3000/statistics')
        
        # 응답 상태 코드 확인
        if response.status_code == 200:
            # 성공적으로 데이터를 받아왔을 때
            data = response.json()
            return jsonify(data), 200
        else:
            # 오류 발생 시
            return jsonify({"error": "외부 API에서 데이터를 가져오는 데 실패했습니다."}), response.status_code
    
    except requests.RequestException as e:
        # 네트워크 오류 등 예외 처리
        return jsonify({"error": f"요청 중 오류 발생: {str(e)}"}), 500

if __name__ == '__main__':
    app.run(debug=True, port=5001)

In [9]:
import json
import os

def load_json_files(directory):
    data = []
    for filename in os.listdir(directory):
        if filename.endswith('.json'):
            with open(os.path.join(directory, filename), 'r', encoding='utf-8') as file:
                data.extend(json.load(file))
    return data

json_data = load_json_files('data/')

In [26]:
import json
from langchain.schema import Document
from pprint import pprint

# JSON 파일 로드 함수 정의
def load_json_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return json.load(file)

# JSON 데이터 로드
data = load_json_file("data/dummy.json")

# 필요한 정보 추출 및 Document 객체 생성
docs = [
    Document(
        page_content=item["text"],
        metadata={
            "id": item["id"],
            "emoji": item["metadata"].get("emoji"),
            "miniuteDuration": item["metadata"].get("miniuteDuration")
        }
    )
    for item in data
]

# 결과 출력
pprint(docs)

[Document(metadata={'id': '1', 'emoji': '🏃', 'miniuteDuration': 5}, page_content='가볍게 5분 걷기'),
 Document(metadata={'id': '2', 'emoji': '🧘', 'miniuteDuration': 5}, page_content='스트레칭으로 몸 풀기'),
 Document(metadata={'id': '3', 'emoji': '💪', 'miniuteDuration': 3}, page_content='팔굽혀펴기 10회'),
 Document(metadata={'id': '4', 'emoji': '🦵', 'miniuteDuration': 5}, page_content='스쿼트 15회'),
 Document(metadata={'id': '5', 'emoji': '🚶', 'miniuteDuration': 10}, page_content='10분 걷기로 마무리'),
 Document(metadata={'id': '6', 'emoji': '🧘\u200d♂️', 'miniuteDuration': 15}, page_content='요가로 몸과 마음 안정시키기'),
 Document(metadata={'id': '7', 'emoji': '🚴', 'miniuteDuration': 20}, page_content='자전거 타기 20분'),
 Document(metadata={'id': '8', 'emoji': '🤸', 'miniuteDuration': 2}, page_content='플랭크 1분 유지'),
 Document(metadata={'id': '9', 'emoji': '🦵', 'miniuteDuration': 5}, page_content='런지 10회'),
 Document(metadata={'id': '10', 'emoji': '🏃\u200d♂️', 'miniuteDuration': 15}, page_content='가벼운 조깅 15분')]


In [32]:
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_upstage import UpstageEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate

# 1. 벡터 저장소 생성
passage_embeddings = UpstageEmbeddings(model="solar-embedding-1-large-query")
vectorstore = Chroma(embedding_function=passage_embeddings)


In [33]:
from langchain.storage import LocalFileStore
from langchain.embeddings import CacheBackedEmbeddings

fs = LocalFileStore("./cache/")
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    passage_embeddings, fs, namespace=passage_embeddings.model
)

# 2. 문서 임베딩 및 저장
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(docs)
vectorstore.add_documents(texts)


['6eb02a76-6a6e-42b8-ad53-bff4a3395cdb',
 '9329517f-9749-48de-b850-1358193782cc',
 '2435c6d4-e3fe-4fb1-a251-04ddccfeece9',
 'd70f27f9-938a-4177-8937-58624d8c42d8',
 'e06d786d-2e2f-4cf6-a06b-e7c814bb9566',
 'f2f9fdfd-4a51-470d-95cb-e2e6a753779b',
 'dd4b6b7e-177d-4124-94f3-18d8c7f14137',
 '2f88e5e6-d301-4135-bd62-c38281e6e783',
 '76b3af85-ba58-4df9-b601-326d57ed9bcd',
 'fc15a954-f191-4a57-903c-d34b557c0f2c']

In [42]:
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# 3. 검색 기능 구현
retriever = vectorstore.as_retriever()

# 4. LLM 설정 (OpenAI API 키가 필요합니다)
model = ChatOpenAI(temperature=0, model_name="gpt-4o-mini-2024-07-18")

# 더미 데이터 생성
dummy_data = {
    "context": "운동 루틴에 대한 정보가 포함된 문서 내용입니다.",
    "question": "운동 루틴에 대해 설명해주세요.",
    "age": 24,
    "gender": "남성",
    "Job": "학생",
    "Difficult": "의욕 떨어짐, 목표 너무 큼",
    "goal": "꾸준히 운동하기"
}

# 5. RAG 파이프라인 구축
template = """당신은 API 서버입니다.
{context}
사용자의 나이는 {age}입니다.
사용자의 성별은 {gender}입니다.
사용자의 직업은 {Job}입니다.
사용자의 겪고 있는 어려움은 {Difficult}입니다.
사용자의 목표는 {goal}입니다.

질문: {question}
답변:"""

prompt = PromptTemplate(
    template=template,
    input_variables=["context", "question", "age", "gender", "Job", "Difficult", "goal"]
)

# RAG 체인 생성
chain = (
     prompt
    | model
    | StrOutputParser()
)

# 결과 생성
result = chain.invoke(dummy_data)

# 결과 출력
print(result)

운동 루틴은 개인의 목표와 상황에 맞춰 설계된 운동 계획입니다. 24세 남성 학생인 당신의 경우, 꾸준히 운동하기 위한 루틴을 제안드립니다. 의욕이 떨어지거나 목표가 너무 크다고 느낀다면, 작은 목표를 설정하고 점진적으로 발전하는 것이 중요합니다.

### 운동 루틴 제안

1. **주 3-4회 운동**: 매일 운동하기보다는 주 3-4회로 시작하여 몸이 적응할 수 있도록 합니다.

2. **운동 시간**: 한 번에 30-45분 정도 운동하는 것을 목표로 합니다. 짧은 시간 동안 집중적으로 운동하는 것이 의욕을 유지하는 데 도움이 됩니다.

3. **운동 종류**:
   - **유산소 운동**: 조깅, 자전거 타기, 수영 등. 주 2-3회, 20-30분 정도.
   - **근력 운동**: 체중을 이용한 운동(푸시업, 스쿼트 등)이나 가벼운 덤벨을 이용한 운동. 주 2회, 20-30분 정도.
   - **스트레칭**: 운동 전후로 5-10분간 스트레칭을 통해 부상을 예방하고 유연성을 높입니다.

4. **목표 설정**: 처음에는 '주 3회 운동하기'와 같은 작은 목표를 설정하고, 이를 달성했을 때 스스로에게 보상을 주는 방식으로 동기를 부여합니다.

5. **운동 파트너**: 친구나 동료와 함께 운동하면 서로의 의욕을 북돋아 줄 수 있습니다.

6. **일지 작성**: 운동 일지를 작성하여 자신의 진행 상황을 기록하고, 성취감을 느낄 수 있도록 합니다.

이러한 루틴을 통해 꾸준히 운동하는 습관을 기르고, 점차 목표를 확장해 나갈 수 있습니다. 중요한 것은 자신에게 맞는 속도로 진행하며, 즐겁게 운동하는 것입니다.
